## Check county names
- `cn`: `County Name` value
- `ocn`: county name implied by "original" `Latitude`/`Longitude` fields (renamed `olat`/`olon` here)
- `icn`: county name implied by "interpolated" lat/lon (`ilat`/`ilon`) value

In [1]:
from utz import *
from njdot.crashes import Crashes, pk_cols, ksi_renames
from njdot.data import cc2cn, cn2cc
from njdot.paths import CNS
from nj_crashes.paths import PUBLIC_DIR

In [2]:
%%time
cns = read_parquet(CNS, columns=['cn', 'icn', 'ocn'])
cns

CPU times: user 406 ms, sys: 70.6 ms, total: 476 ms
Wall time: 255 ms


,cn,icn,ocn
id,,,
0,Atlantic,None,None
1,Atlantic,None,None
2,Atlantic,None,None
3,Atlantic,Atlantic,None
4,Atlantic,None,None
...,...,...,...
6077298,Warren,Warren,Warren
6077299,Warren,Warren,Warren
6077300,Warren,Warren,Warren


In [3]:
cn_ocn = cns.ocn.copy().rename('cn_ocn').fillna('nan')
ocn_isn = ~cns.ocn.isna()
cn_ocn.loc[ocn_isn & (cns.ocn == cns.cn)] = 'eq'
cn_ocn.loc[ocn_isn & (cns.ocn != cns.cn)] = 'neq'

In [4]:
cn_icn = cns.icn.copy().rename('cn_icn').fillna('nan')
icn_isn = ~cns.icn.isna()
cn_icn.loc[icn_isn & (cns.icn == cns.cn)] = 'eq'
cn_icn.loc[icn_isn & (cns.icn != cns.cn)] = 'neq'

In [5]:
names = [ 'nan', 'eq', 'neq' ]
ct = pd.crosstab(cn_ocn, cn_icn).loc[names, names]
n = ct.sum().sum()
ct.loc['sum'] = ct.sum(0)
ct['sum'] = ct.sum(1)
ct

cn_icn,nan,eq,neq,sum
cn_ocn,,,,
nan,2505195,2297595,22867,4825657
eq,90888,1134018,5381,1230287
neq,1478,15327,4554,21359
sum,2597561,3446940,32802,6077303


In [6]:
pct = round(ct / n * 100, 1)
pct

cn_icn,nan,eq,neq,sum
cn_ocn,,,,
nan,41.2,37.8,0.4,79.4
eq,1.5,18.7,0.1,20.2
neq,0.0,0.3,0.1,0.4
sum,42.7,56.7,0.5,100.0


In crashes where:
- "original lat/lon" and and "interpolated lat/lon" county names both exist, and
- both don't match the provided `County Name`,

Do they at least match each other?

In [7]:
neqs = cns[(cn_icn == 'neq') & (cn_ocn == 'neq')]
neqs_eq = neqs.icn == neqs.ocn
neqs_eq.value_counts()

True     4445
False     109
Name: count, dtype: int64

Crashes with 3 different county names / locations 😵‍💫

In [8]:
cns[['cn', 'icn', 'ocn']].loc[neqs_eq[~neqs_eq].index].value_counts().sort_index()

cn          icn         ocn       
Bergen      Hudson      Essex          2
            Passaic     Essex          1
                        Monmouth       1
Burlington  Camden      Cumberland     1
                        Gloucester     1
                        Ocean          1
Camden      Burlington  Gloucester    10
                        Salem          2
            Gloucester  Atlantic       2
                        Cumberland     1
                        Salem          1
Cape May    Atlantic    Burlington     1
                        Cumberland     2
                        Ocean          1
Cumberland  Salem       Gloucester     2
Essex       Hudson      Hunterdon      1
                        Ocean          1
            Morris      Somerset       1
                        Union          1
            Union       Middlesex      3
                        Morris        12
                        Somerset       2
Gloucester  Camden      Atlantic       7
                      